<a href="https://colab.research.google.com/github/Austin-Cyr/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,430 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http:

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-03 04:24:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-03 04:24:16 (6.62 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

In [15]:
vine20_df = vine_df.filter("total_votes>=20")
vine20_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R14G4YQXL17REH|          1|            4|         28|   N|     

In [31]:
vinehv_tv = vine20_df.withColumn('HV_TV', vine20_df['helpful_votes'] / vine20_df['total_votes'])
vinehv_tv.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|              HV_TV|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N| 0.6923076923076923|
| R7K83D59T9KWS|          1|           16|         26|   N|                N| 0.6153846153846154|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|               0.64|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N| 0.9344262295081968|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|                1.0|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y| 0.9072164948453608|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y| 0.6744186046511628|
|R39OOYN0DOLINK|    

In [32]:
vinehv50 = vinehv_tv.filter('HV_TV>=.50')
vinehv50.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             HV_TV|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|0.6923076923076923|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|0.6153846153846154|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|              0.64|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|0.9344262295081968|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|               1.0|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|0.9072164948453608|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|0.6744186046511628|
|R39OOYN0DOLINK|          3|  

In [38]:
vine_paid = vinehv50.filter("vine== 'Y'")
vine_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             HV_TV|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R28EIR3Z6RRUNN|          2|           50|         60|   Y|                N|0.8333333333333334|
|R2YO2RESXZEWOF|          4|           24|         25|   Y|                N|              0.96|
| R8GEE2EW3D6S9|          3|           49|         54|   Y|                N|0.9074074074074074|
|R33W9V1MX6WKPS|          4|           47|         53|   Y|                N|0.8867924528301887|
|R24E3W2ZRIFG8C|          5|           91|         93|   Y|                N| 0.978494623655914|
|R2OR3KIFFG4YWT|          4|           75|         81|   Y|                Y|0.9259259259259259|
|R1RLJAAFOA5MMS|          2|           25|         27|   Y|                N|0.9259259259259259|
| RVYHD7IZ7W99B|          1|  

In [39]:
vine_notpaid = vinehv50.filter("vine== 'N'")
vine_notpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             HV_TV|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|0.6923076923076923|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|0.6153846153846154|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|              0.64|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|0.9344262295081968|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|               1.0|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|0.9072164948453608|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|0.6744186046511628|
|R39OOYN0DOLINK|          3|  

Total Paid Vine Reviews

In [50]:
vine_paid.count()


248

Number of 5 Star Review Products with Paid Vine Reviews

In [42]:
vine_paid.filter("star_rating == '5'").count()

102

Percent of Paid Reviews with 5 Star Product Ratings

In [43]:
102/248

0.4112903225806452

Total Products without Paid Vine Reviews

In [44]:
vine_notpaid.count()

17514

Number of 5 Star Review Products without Paid Vine Reviews

In [45]:
vine_notpaid.filter("star_rating == '5'").count()

5154

Percent of Not Paid Reviews with 5 Star Product Ratings

In [46]:
5154/17514

0.29427886262418634